In [37]:
'''
Sample code to read dicom images and place slices as channels of RGB
'''
import matplotlib.pyplot as plt
import numpy as np
import os
import pydicom
from pydicom.data import get_testdata_files
from PIL import Image #for RGB manipulation
from medpy.io import load #for mha files

#image_data, image_header = load('/path/to/image.mha')

filename = get_testdata_files("rtplan.dcm")[0]
ds = pydicom.dcmread(filename)  # plan dicom_data
ds.PatientName
setup_names = ds.dir("setup")

print(len(setup_names))
print(type(setup_names))
setup_names


#
#sample images
#
#/home/kgonzalez/CANCER_IMAGE_FILES/TEST_SET/NSCLC Radiogenomics-Demo/GSM714049/10-11-1990-PET CT LUNG CANCER-99045/2-CT Images-62820/

dicom_dir = '/home/kgonzalez/CANCER_IMAGE_FILES/TEST_SET/NSCLC Radiogenomics-Demo/GSM714049/10-11-1990-PET CT LUNG CANCER-99045/2-CT Images-62820/'

image_dir = '/home/kgonzalez/COMSCI_168_SPRING_2019/Project/RGB'



###########################################################################
# Dicom top level for RCC data. We will need to drill down into this to get the individual patient records
#
clear_cell_pt_dir = '/media/kgonzalez/Seagate Backup Plus Drive/Heidi/clear_cell_onco/clear_cell'

#the clear cell directory is 
#  top
#     -> patient directories\
#          -> PreContrast\
#              -> images\
#              -> segmentation files (.mha,.roi)
#          -> post contrast nephro\
#          -> post coricomedullary\
#          -> post excretory\




#get list of patient directories underneath this
patient_directories = sorted(os.listdir(clear_cell_pt_dir))
clear_cell_dict ={}
#for every patient, go into the folder and 

ct_type= [] #list to hold the sub directories
counter = 0
print(type(patient_directories))
for ii in patient_directories:
    #print(ii)
    #get list of 4 sub directories
    ct_type.append(os.listdir(os.path.join(clear_cell_pt_dir,ii)))
    print('<>',ii,(os.listdir(os.path.join(clear_cell_pt_dir,ii))))
    #store the ct type listing folders under the patient number key
    clear_cell_dict[ii] = os.listdir(os.path.join(clear_cell_pt_dir,ii))
    
    
    



print(ct_type[0])


#for every patient, they have 4 folders of different tests conducted. Each folder will have the segmentation
#data in an .mha file

counter = 0
for ii in patient_directories: #ct_type:

    for jj in ct_type[counter]:
        print(ii)
        #get listing in each of the test type directories
        print("----->",clear_cell_pt_dir,ii,jj,counter)
        print(ct_type[counter])
        files_found = os.listdir(os.path.join(clear_cell_pt_dir,ii,jj))
        print(files_found)

    counter = counter + 1

    
#
# Get the image files in each directory
#
image_dict={}
for key, value in clear_cell_dict.items():
    print(key,value)
    
    #first CT directory for debugging
    image_dir = os.path.join(clear_cell_pt_dir,key,value[0])
    print('image_dir ---->',image_dir)
    image_dir_listing = os.listdir(image_dir)
    image_files = os.listdir(os.path.join(image_dir,'images'))
    print(image_files)
    image_dict[key] = image_files
    


print('example  ',list(image_dict.keys())[0])
image1 = list(image_dict.key())[0]
full_name = os.path.join(clear_cell_pt_dir,clear_cell_dict[image1],'images',image1)
print(full_name)
return



files = sorted(os.listdir(dicom_dir))
#files=files.sort
print(len(files))
print(type(files))

position = {}
file_dict = {}

for ii in files[0:20]:
    full_file = os.path.join(dicom_dir, ii)
    #full_file = os.join(dicom_dir,ii)
    dicom_data = pydicom.dcmread(full_file)
    # Normal mode:
    print()
    print("Filename.........:", full_file)
    print("Storage type.....:", dicom_data.SOPClassUID)
    print()
    
    pat_name = dicom_data.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print("Patient's name...:", display_name)
    print("Patient id.......:", dicom_data.PatientID)
    print("Modality.........:", dicom_data.Modality)
    print("Study Date.......:", dicom_data.StudyDate)
    
    #check to make sure raw data is in the file
    if 'PixelData' in dicom_data:
        rows = int(dicom_data.Rows)
        cols = int(dicom_data.Columns)
        print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
        rows=rows, cols=cols, size=len(dicom_data.PixelData)))
    if 'PixelSpacing' in dicom_data:
        print("Pixel spacing....:", dicom_data.PixelSpacing)

# use .get() if not sure the item exists, and want a default value if missing
    print("Slice location...:", dicom_data.get('SliceLocation', "(missing)"))

    
    #assign slice position
    location = dicom_data.get('SliceLocation', "(missing)")
    position[ii] = location
    file_dict[ii]= full_file
    
    
    

# plot the image using matplotlib
    #plt.figure()
    #plt.imshow(dicom_data.pixel_array, cmap='bone')
    #plt.colorbar()
    #plt.show()
    
#sort dictionary to get the slice order
newpos = {}

for key, value in sorted(position.items(), key=lambda item: item[1]):
    #print("%s: %s" % (key, value))
    newpos[key] = value
    

    
for ii in range(0,len(position)-1):
    #slice = list(newpos.keys())[ii]
    #print(slice)
    #a = next(iter(newpos.values()))
    #print(a)
    
    slice = list(newpos.keys())[ii-1]
    print(slice)
    print(newpos[slice])
    full_file_r = os.path.join(dicom_dir, slice)
    slice = list(newpos.keys())[ii]
    full_file_g = os.path.join(dicom_dir, slice)
    slice = list(newpos.keys())[ii+1]
    full_file_b = os.path.join(dicom_dir, slice)
    
    dicom_data_r = pydicom.dcmread(full_file_r)
    dicom_data_g = pydicom.dcmread(full_file_g)
    dicom_data_b = pydicom.dcmread(full_file_b)
    
    #(dicom_data.PixelData)
    #full_file = os.join(dicom_dir,ii)
    #dicom_data = pydicom.dcmread(full_file)
    # r, g, and b are 512x512 float arrays with values >= 0 and < 1.
    rgbArray = np.zeros((cols,rows,3), 'uint8')
    print(len(dicom_data_g.PixelData))
    
    plt.figure()
    plt.imshow(dicom_data_g.pixel_array)
    
    rgbArray[:,:, 0] = dicom_data_r.pixel_array
    rgbArray[:,:, 1] = dicom_data_g.pixel_array
    rgbArray[..., 2] = dicom_data_b.pixel_array
    img = Image.fromarray(rgbArray)
    
    rgb_file = os.path.join(image_dir, 'rgb' + str(ii) + '.png')
    img.save(rgb_file)
    
    #position[ii]
return


1
<class 'list'>
<class 'list'>
<> 10024-0011_2003-05-19 ['1.2.840.113619.2.30.1.1762269721.1947.1053353574.556-CT Pre-Contrast', '1.2.840.113619.2.30.1.1762269721.1947.1053353574.617-CT Post-Contrast Coricomedullary', '1.2.840.113619.2.30.1.1762269721.1947.1053353574.617-CT Post-Contrast Excretor', '1.2.840.113619.2.30.1.1762269721.1947.1053353574.617-CT Post-Contrast Nephrographic']
<> 10024-0012_2003-03-27 ['1.2.840.113619.2.30.1.1762532635.1940.1048796097.300-CT Pre-Contrast', '1.2.840.113619.2.30.1.1762532635.1940.1048796097.361-CT Post-Contrast Coricomedullary', '1.2.840.113619.2.30.1.1762532635.1940.1048796097.361-CT Post-Contrast Excretor', '1.2.840.113619.2.30.1.1762532635.1940.1048796097.361-CT Post-Contrast Nephrographic']
<> 10024-0013_2005-09-21 ['1.3.12.2.1107.5.1.4.50146.30000005092112012459300000122-CT Post-Contrast Coricomedullary', '1.3.12.2.1107.5.1.4.50146.30000005092112012459300000124-CT Post-Contrast Nephrographic', '1.3.12.2.1107.5.1.4.50146.300000050921151452234

['1000089119.mha', '1000089119.roi', '10024-0029.seri', 'images']
10024-0030_2003-10-29
-----> /media/kgonzalez/Seagate Backup Plus Drive/Heidi/clear_cell_onco/clear_cell 10024-0030_2003-10-29 1.2.840.113619.2.30.1.1762342870.1892.1067449406.354-CT Pre-Contrast 18
['1.2.840.113619.2.30.1.1762342870.1892.1067449406.354-CT Pre-Contrast', '1.2.840.113619.2.30.1.1762342870.1892.1067449406.390-CT Post-Contrast Coricomedullary', '1.2.840.113619.2.30.1.1762342870.1892.1067449406.435-CT Post-Contrast Nephrographic', '1.2.840.113619.2.30.1.1762342870.1892.1067449406.558-CT Post-Contrast Excretor']
['1000089812.mha', '1000089812.roi', '10024-0030.seri', 'images']
10024-0030_2003-10-29
-----> /media/kgonzalez/Seagate Backup Plus Drive/Heidi/clear_cell_onco/clear_cell 10024-0030_2003-10-29 1.2.840.113619.2.30.1.1762342870.1892.1067449406.390-CT Post-Contrast Coricomedullary 18
['1.2.840.113619.2.30.1.1762342870.1892.1067449406.354-CT Pre-Contrast', '1.2.840.113619.2.30.1.1762342870.1892.1067449406

['1000136522.mha', '1000136522.roi', '10024-0358.seri', 'images']
10024-0358_2011-08-01
-----> /media/kgonzalez/Seagate Backup Plus Drive/Heidi/clear_cell_onco/clear_cell 10024-0358_2011-08-01 1.3.12.2.1107.5.1.4.55174.30000011080113510717100002573-CT Post-Contrast Excretor 80
['1.3.12.2.1107.5.1.4.55174.30000011080113510717100002136-CT Pre-Contrast', '1.3.12.2.1107.5.1.4.55174.30000011080113510717100002232-CT Post-Contrast Coricomedullary', '1.3.12.2.1107.5.1.4.55174.30000011080113510717100002324-CT Post-Contrast Nephrographic', '1.3.12.2.1107.5.1.4.55174.30000011080113510717100002573-CT Post-Contrast Excretor']
['1000136522.mha', '1000136522.roi', '10024-0358.seri', 'images']
10024-0359_2010-08-23
-----> /media/kgonzalez/Seagate Backup Plus Drive/Heidi/clear_cell_onco/clear_cell 10024-0359_2010-08-23 1.3.12.2.1107.5.1.4.55034.30000010082314422064000003437-CT Pre-Contrast 81
['1.3.12.2.1107.5.1.4.55034.30000010082314422064000003437-CT Pre-Contrast', '1.3.12.2.1107.5.1.4.55034.30000010

['1.3.12.2.1107.5.1.4.60034.30000010012115243637500015051', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015052', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015053', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015054', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015055', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015056', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015057', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015058', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015059', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015060', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015061', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015062', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015063', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015064', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015065', '1.3.12.2.1107.5.1.4.60034.30000010012115243637500015066', '1.3.12.2.1107.5.1.4.60034.3000001001211524363750001506

AttributeError: 'dict' object has no attribute 'key'

In [24]:
clear_cell_dict

{'10024-0011_2003-05-19': ['1.2.840.113619.2.30.1.1762269721.1947.1053353574.556-CT Pre-Contrast',
  '1.2.840.113619.2.30.1.1762269721.1947.1053353574.617-CT Post-Contrast Coricomedullary',
  '1.2.840.113619.2.30.1.1762269721.1947.1053353574.617-CT Post-Contrast Excretor',
  '1.2.840.113619.2.30.1.1762269721.1947.1053353574.617-CT Post-Contrast Nephrographic'],
 '10024-0012_2003-03-27': ['1.2.840.113619.2.30.1.1762532635.1940.1048796097.300-CT Pre-Contrast',
  '1.2.840.113619.2.30.1.1762532635.1940.1048796097.361-CT Post-Contrast Coricomedullary',
  '1.2.840.113619.2.30.1.1762532635.1940.1048796097.361-CT Post-Contrast Excretor',
  '1.2.840.113619.2.30.1.1762532635.1940.1048796097.361-CT Post-Contrast Nephrographic'],
 '10024-0013_2005-09-21': ['1.3.12.2.1107.5.1.4.50146.30000005092112012459300000122-CT Post-Contrast Coricomedullary',
  '1.3.12.2.1107.5.1.4.50146.30000005092112012459300000124-CT Post-Contrast Nephrographic',
  '1.3.12.2.1107.5.1.4.50146.30000005092115145223400000000-CT